# Two-Tone Qubit Spectroscopy Demo

This notebook demonstrates two-tone spectroscopy experiments which apply two frequency-swept tones simultaneously to probe:
- Multi-photon transitions
- Sideband effects  
- Qubit-resonator coupling
- AC Stark shifts

## 1. Setup Simulation Environment

In [ ]:
import numpy as np
from leeq.experiments.experiments import ExperimentManager
from leeq.experiments.builtin.basic.calibrations.two_tone_spectroscopy import TwoToneQubitSpectroscopy
from leeq.core.elements.built_in.qudit_transmon import TransmonElement
from leeq.setups.built_in.setup_simulation_high_level import HighLevelSimulationSetup
from leeq.theory.simulation.numpy.rotated_frame_simulator import VirtualTransmon
from leeq.chronicle import Chronicle

# Start logging
Chronicle().start_log()

## 2. Configure Virtual Qubit and Setup

In [ ]:
# Clear any existing setups
manager = ExperimentManager()
manager.clear_setups()

# Create virtual transmon with realistic parameters
virtual_transmon = VirtualTransmon(
    name="TestQubit",
    qubit_frequency=5000.0,  # MHz
    anharmonicity=-200.0,    # MHz
    t1=50,                   # us
    t2=25,                   # us
    readout_frequency=9500.0, # MHz
    quiescent_state_distribution=np.array([0.9, 0.08, 0.02, 0.0])
)

# For two-tone on different channels, we need a second virtual qubit
virtual_transmon2 = VirtualTransmon(
    name="TestQubit2",
    qubit_frequency=4800.0,  # Different frequency for f12 transition
    anharmonicity=-200.0,
    t1=50,
    t2=25,
    readout_frequency=9500.0,
    quiescent_state_distribution=np.array([0.9, 0.08, 0.02, 0.0])
)

# Setup high-level simulation
setup = HighLevelSimulationSetup(
    name='TwoToneDemo',
    virtual_qubits={
        1: virtual_transmon,   # Channel 1 (f01)
        2: virtual_transmon2,  # Channel 2 (f12)
        3: virtual_transmon    # Channel 3 (readout)
    }
)

# Optional: Set coupling between qubits
setup.set_coupling_strength_by_qubit(virtual_transmon, virtual_transmon2, coupling_strength=2.0)

manager.register_setup(setup)
print("Setup registered successfully")

## 3. Configure Qubit Element

In [ ]:
# Define qubit configuration
qubit_config = {
    'lpb_collections': {
        'f01': {
            'type': 'SimpleDriveCollection',
            'freq': 5000.0,
            'channel': 1,
            'shape': 'blackman_drag',
            'amp': 0.5,
            'phase': 0.,
            'width': 0.05,
            'alpha': 500,
            'trunc': 1.2
        },
        'f12': {
            'type': 'SimpleDriveCollection',
            'freq': 4800.0,  # f01 - anharmonicity
            'channel': 2,
            'shape': 'blackman_drag',
            'amp': 0.1,
            'phase': 0.,
            'width': 0.025,
            'alpha': 425,
            'trunc': 1.2
        }
    },
    'measurement_primitives': {
        '0': {
            'type': 'SimpleDispersiveMeasurement',
            'freq': 9500.0,
            'channel': 3,
            'shape': 'square',
            'amp': 0.15,
            'phase': 0.,
            'width': 1,
            'trunc': 1.2,
            'distinguishable_states': [0, 1]
        }
    }
}

# Create qubit element
qubit = TransmonElement(name='Q1', parameters=qubit_config)
print(f"Created qubit element: {qubit.hrid}")

## 4. Two-Tone Spectroscopy - Different Channels

First, we'll run two-tone spectroscopy with the two tones on different channels (f01 and f12 transitions).

In [ ]:
# Run two-tone spectroscopy on different channels
# Constructor automatically runs the experiment
exp_different = TwoToneQubitSpectroscopy(
    dut_qubit=qubit,
    tone1_start=4950.0,
    tone1_stop=5050.0,
    tone1_step=5.0,
    tone1_amp=0.1,
    tone2_start=4750.0,
    tone2_stop=4850.0,
    tone2_step=5.0,
    tone2_amp=0.1,
    same_channel=False,  # Different channels
    num_avs=1000,
    mp_width=1.0
)

# Plots are displayed automatically when experiment finishes
print(f"Experiment completed. Data shape: {exp_different.result['Magnitude'].shape}")

## 5. Two-Tone Spectroscopy - Same Channel

Now we'll run two-tone spectroscopy with both tones superimposed on the same channel.

In [ ]:
# Run two-tone spectroscopy on same channel
exp_same = TwoToneQubitSpectroscopy(
    dut_qubit=qubit,
    tone1_start=4980.0,
    tone1_stop=5020.0,
    tone1_step=2.0,
    tone1_amp=0.05,
    tone2_start=5000.0,
    tone2_stop=5040.0,
    tone2_step=2.0,
    tone2_amp=0.05,
    same_channel=True,  # Same channel (superimposed)
    num_avs=500
)

print(f"Experiment completed. Data shape: {exp_same.result['Magnitude'].shape}")

## 6. Analysis: Peak Detection

In [ ]:
# Find resonance peaks
peaks_different = exp_different.find_peaks()
peaks_same = exp_same.find_peaks()

print("Different Channels Peak Detection:")
print(f"  Tone 1 Peak: {peaks_different['peak_freq1']:.1f} MHz")
print(f"  Tone 2 Peak: {peaks_different['peak_freq2']:.1f} MHz")
print(f"  Peak Magnitude: {peaks_different['peak_magnitude']:.4f}")

print("\nSame Channel Peak Detection:")
print(f"  Tone 1 Peak: {peaks_same['peak_freq1']:.1f} MHz")
print(f"  Tone 2 Peak: {peaks_same['peak_freq2']:.1f} MHz")
print(f"  Peak Magnitude: {peaks_same['peak_magnitude']:.4f}")

## 7. Analysis: Cross-Sections

Extract 1D cross-sections from the 2D data to examine specific frequency slices.

In [ ]:
import plotly.graph_objects as go

# Get cross-sections at peak locations
cross_freq1 = exp_different.get_cross_section(axis='freq1')
cross_freq2 = exp_different.get_cross_section(axis='freq2')

# Plot cross-sections
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=cross_freq1['frequencies'],
    y=cross_freq1['magnitude'],
    mode='lines+markers',
    name=f"Fixed Tone1={cross_freq1['slice_freq']:.1f} MHz"
))

fig.add_trace(go.Scatter(
    x=cross_freq2['frequencies'],
    y=cross_freq2['magnitude'],
    mode='lines+markers',
    name=f"Fixed Tone2={cross_freq2['slice_freq']:.1f} MHz"
))

fig.update_layout(
    title="Two-Tone Spectroscopy Cross-Sections",
    xaxis_title="Frequency (MHz)",
    yaxis_title="Magnitude",
    width=800,
    height=400
)

fig.show()

## 8. Phase Analysis

The phase information can reveal additional physics such as dispersive shifts.

In [ ]:
# Calculate phase gradient to identify regions of strong interaction
phase = exp_different.result['Phase']
phase_gradient_x = np.gradient(phase, axis=0)
phase_gradient_y = np.gradient(phase, axis=1)
phase_gradient_mag = np.sqrt(phase_gradient_x**2 + phase_gradient_y**2)

# Plot phase gradient magnitude
fig = go.Figure(data=go.Heatmap(
    x=exp_different.freq2_arr,
    y=exp_different.freq1_arr,
    z=phase_gradient_mag,
    colorscale='Viridis',
    colorbar=dict(title='Phase Gradient')
))

fig.update_layout(
    title='Phase Gradient Magnitude - Identifies Strong Interactions',
    xaxis_title='Tone 2 Frequency (MHz)',
    yaxis_title='Tone 1 Frequency (MHz)',
    width=800,
    height=600
)

fig.show()

# Find maximum phase gradient location
max_grad_idx = np.unravel_index(np.argmax(phase_gradient_mag), phase_gradient_mag.shape)
print(f"Maximum phase gradient at:")
print(f"  Tone 1: {exp_different.freq1_arr[max_grad_idx[0]]:.1f} MHz")
print(f"  Tone 2: {exp_different.freq2_arr[max_grad_idx[1]]:.1f} MHz")

## 9. Power Dependence Study

Study how the two-tone response changes with drive amplitude.

In [ ]:
# Run with different power combinations
amplitudes = [0.05, 0.1, 0.2]
results = []

for amp in amplitudes:
    exp = TwoToneQubitSpectroscopy(
        dut_qubit=qubit,
        tone1_start=4990.0,
        tone1_stop=5010.0,
        tone1_step=5.0,
        tone1_amp=amp,
        tone2_start=4790.0,
        tone2_stop=4810.0,
        tone2_step=5.0,
        tone2_amp=amp,
        same_channel=False,
        num_avs=200
    )
    peaks = exp.find_peaks()
    results.append({
        'amp': amp,
        'peak_mag': peaks['peak_magnitude'],
        'freq1': peaks['peak_freq1'],
        'freq2': peaks['peak_freq2']
    })
    print(f"Amplitude {amp:.2f}: Peak magnitude = {peaks['peak_magnitude']:.4f}")

# Plot power dependence
fig = go.Figure()
fig.add_trace(go.Scatter(
    x=[r['amp'] for r in results],
    y=[r['peak_mag'] for r in results],
    mode='lines+markers',
    marker=dict(size=10),
    name='Peak Response'
))

fig.update_layout(
    title='Two-Tone Response vs Drive Amplitude',
    xaxis_title='Drive Amplitude',
    yaxis_title='Peak Magnitude',
    width=600,
    height=400
)

fig.show()

## 10. Summary

This notebook demonstrated:
1. **Two-tone spectroscopy** with dual frequency sweeps
2. **Different channel mode** - probing f01 and f12 transitions simultaneously
3. **Same channel mode** - superimposing two tones on the same drive channel
4. **Peak detection** - automatic identification of resonances
5. **Cross-section analysis** - extracting 1D slices from 2D data
6. **Phase gradient analysis** - identifying regions of strong interaction
7. **Power dependence** - studying amplitude effects

Key observations:
- The experiment class handles both simulation and hardware modes automatically
- Plots are generated automatically via the `@register_browser_function` decorator
- The sweep engine efficiently handles 2D parameter sweeps
- CWSpectroscopySimulator provides realistic multi-tone simulation including crosstalk